In [2]:
from google import genai
import os
from dotenv import load_dotenv
import duckdb
load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

# response = client.models.generate_content(
#     model="gemini-2.0-flash", contents="Ge mig en historia om en kanin som bor i skogen på 3 meningar"
# )
# print(response.text)



In [8]:
with duckdb.connect("../../ads_data.duckdb") as con: 
    df_mart = con.execute("SELECT * FROM mart.mart_ads").df()
df_test = df_mart[df_mart["occupation_field"] == "Kultur, media, design"]
# df_test.to_csv("sample_ads.csv") # måste göra mer finjusteringar! kan inte läsa allas description direkt!


In [10]:
descriptions = df_test["description"]

top_employer = df_test.groupby("employer_name")[["vacancies", "description"]].sum().sort_values(by="vacancies", ascending=False).reset_index()
top_employer
# df_test

,employer_name,vacancies,description
0,Vs Consulting AB,500,Vi är en modern tolkförmedling som bokar och f...
1,Exakta Photo AB,100,"Fotograf, ett drömyrke för dig?\nVi söker nu f..."
2,Stegra Boden AB,74,Research shows that women and other under-repr...
3,Språkservice Sverige AB,64,Språkservice Sverige AB är Sveriges största to...
4,H & M Hennes & Mauritz Gbc AB,48,WHAT YOU’LL DO​ \nAs a Global visual & commerc...
...,...,...,...
229,Zinzino Operations AB,1,Zinzino is a global direct sales company from ...
230,ÖSTRA GÖINGE KOMMUN,1,Östra Göinge kommun med drygt 14 000 invånare ...
231,Örebro Universitet,1,Örebro universitet skapar framtiden genom inte...
232,Öregrund Marine Service AB,1,Har du erfarenhet inom eftermarknad och/eller ...


In [13]:
nr_1 = top_employer[top_employer["employer_name"] == 'Vs Consulting AB']
nr_1["description"].to_csv("top_employer.csv")
# nr_1["description"]

In [10]:
df_test.reset_index()

,index,description
0,0,Tedelektro håller på att expandera och vi behö...
1,1,Lagertekniker (Deltid/Extra) – Crona Software\...
2,2,Vi söker en bilmekaniker som vill jobba anting...
3,3,Är du vår nya verkstadstekniker? Vi expander o...
4,4,Plats: Falköping\nStartdatum: 2025-05-15 \nAns...
...,...,...
3769,5049,ONE Nordic bedriver en samhällsviktig verksamh...
3770,5050,Är du teknikintresserad och vill vara en del a...
3771,5097,Om tjänstenVi söker nu mekaniker till våra kun...
3772,5101,Välj inte Elis om one-man-show är din typ av f...


In [18]:
with open("top_employer.csv", "r", encoding='utf-8') as file: 
    ad_text = file.read() 
# ad_text = nr_1["description"]

promt = f"""Du är en rekryterare inom data Media, Kultur, Design.
Plocka ut 10 av de vanligaste erfarenheterna och kraven som efterfrågas i dessa jobbannonser:
{ad_text}
Output ska vara i detta formatet: 

{{
    "krav": [erfarenhet1, erfarenhet2, ...]
    "meriterande": [meriterande1,meriterande2, ... ]
}}

"""

In [19]:
response = client.models.generate_content(
    model="gemini-2.0-flash", contents=promt
)
print(response.text)

Baserat på de jobbannonser du har tillhandahållit, här är en sammanställning av de 10 vanligaste erfarenheterna och kraven, samt meriterande kvalifikationer, formaterat som du önskat:

```json
{
    "krav": [
        "Mycket goda språkfärdigheter i [specificerat språk] och svenska",
        "God samhällsuppfattning",
        "Serviceinriktad",
        "Förmåga att agera i många miljöer och sammanhang",
        "Hög integritet",
        "Noggrannhet",
        "Mycket ansvarsfull",
        "Punktlighet (kan passa tider)",
        "Förmåga att utföra tolkuppdrag på plats, via telefon och video",
        "Tillgänglighet vid behov"
    ],
    "meriterande": [
        "Auktorisation som tolk",
        "Specialistutbildning inom tolkning",
        "Tolkutbildning",
        "Körkort och tillgång till bil"
    ]
}
```



In [20]:
import pandas as pd
data = response.text
df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in data.items()]))
df

AttributeError: 'str' object has no attribute 'items'